# Principal Components Analysis (PCA)

Task:
- Download the Iris data set.
- Use the handmade pca.py to process data.
- Compare to sklearn version.

Resources:
- [Scikit-learn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
- [Scikit-learn iris data](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)
- [A tutorial on Principal Components Analysis](http://www.cs.otago.ac.nz/cosc453/student_tutorials/principal_components.pdf)


## Imports

In [1]:
# Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Scripts
from pca import TwoDimensionStandardizer, Whitener, PrincipalComponentAnalysis



In [2]:
# TESTING: standardizer

raw_data = np.array([[90, 60, 90], [90, 90, 30], [60, 60, 60]])
np.random.seed(0)
raw_data = np.random.rand(5, 3).round(1)*10

display(raw_data)

my_scaler = TwoDimensionStandardizer()

display(my_scaler.fit_transform(data=raw_data))

skl_scaler = StandardScaler()

display(skl_scaler.fit_transform(raw_data))

scaled_data = skl_scaler.fit_transform(raw_data)


array([[ 5.,  7.,  6.],
       [ 5.,  4.,  6.],
       [ 4.,  9., 10.],
       [ 4.,  8.,  5.],
       [ 6.,  9.,  1.]])

array([[ 0.26726124, -0.21566555,  0.1393466 ],
       [ 0.26726124, -1.83315714,  0.1393466 ],
       [-1.06904497,  0.86266219,  1.53281263],
       [-1.06904497,  0.32349832, -0.2090199 ],
       [ 1.60356745,  0.86266219, -1.60248593]])

array([[ 0.26726124, -0.21566555,  0.1393466 ],
       [ 0.26726124, -1.83315714,  0.1393466 ],
       [-1.06904497,  0.86266219,  1.53281263],
       [-1.06904497,  0.32349832, -0.2090199 ],
       [ 1.60356745,  0.86266219, -1.60248593]])

In [3]:
# TESTING: whitener


def whiten(X, fudge=1E-18):
    """
    Args:
        X (np.array): The matrix X should be observations-by-components
        fudge (float): A fudge factor can be used so that eigenvectors associated with small eigenvalues do not get overamplified.
    """
    # get the covariance matrix
    Xcov = np.dot(X.T,X)
    # eigenvalue decomposition of the covariance matrix
    d, V = np.linalg.eigh(Xcov)
    D = np.diag(1. / np.sqrt(d + fudge))
    # whitening matrix
    W = np.dot(np.dot(V, D), V.T)
    # multiply by the whitening matrix
    X_white = np.dot(X, W)
    return X_white, W


def svd_whiten(X):
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    # U and Vt are the singular matrices, and s contains the singular values.
    # Since the rows of both U and Vt are orthonormal vectors, then U * Vt
    # will be white
    X_white = np.dot(U, Vt)
    return X_white


def my_whitener(data):
    """Input 2D data set. Output whiten & inv. whiten functions. Taken from colab week 2."""
    sigma = np.cov(data, rowvar=False)
    mu = np.mean(data, axis=0)
    values, vectors = np.linalg.eig(sigma)
    l = np.diag(values ** -0.5)

    def func(datapoints):
        return np.array( [ l.dot(vectors.T.dot(d - mu)) for d in datapoints ])

    def inv_func(datapoints):
        return np.array( [ np.linalg.inv(vectors.T).dot(np.linalg.inv(l).dot(d)) + mu for d in datapoints ] )
  
    return func, inv_func


class_whiten = Whitener()
func_whiten, inv_whiten = my_whitener(data=scaled_data)

display(
    '------------------------------------',
    whiten(X=scaled_data)[0],
    whiten(X=scaled_data)[1],
    svd_whiten(X=scaled_data),
    '------------------------------------',
    class_whiten.fit_transform(scaled_data),
    func_whiten(scaled_data),
)


'------------------------------------'

array([[ 0.20979149, -0.074787  ,  0.16701627],
       [ 0.11625556, -0.81144961,  0.07439208],
       [-0.15701342,  0.41884136,  0.71408592],
       [-0.75665251,  0.0735427 , -0.46858132],
       [ 0.58761887,  0.39385255, -0.48691296]])

array([[0.66147306, 0.05782777, 0.3263577 ],
       [0.05782777, 0.4554352 , 0.05726409],
       [0.3263577 , 0.05726409, 0.6612533 ]])

array([[ 0.20979149, -0.074787  ,  0.16701627],
       [ 0.11625556, -0.81144961,  0.07439208],
       [-0.15701342,  0.41884136,  0.71408592],
       [-0.75665251,  0.0735427 , -0.46858132],
       [ 0.58761887,  0.39385255, -0.48691296]])

'------------------------------------'

(array([[ 0.50493456,  0.06115708,  0.22649702],
        [ 0.02674971,  0.06493916,  1.6447089 ],
        [ 0.9026903 , -1.23462617, -0.70770647],
        [-1.69206178, -0.40836727, -0.40020384],
        [ 0.2576872 ,  1.5168972 , -0.76329561]]),
 array([[-0.1181363 , -0.1345709 ,  0.33780327],
        [-1.82563382, -0.1261459 ,  2.04438996],
        [ 0.71318499, -1.7934905 , -1.09893348],
        [-0.056514  ,  0.15246293, -0.74030426],
        [ 1.28709913,  1.90174438, -0.5429555 ]]))

array([[ 0.50493456,  0.06115708,  0.22649702],
       [ 0.02674971,  0.06493916,  1.6447089 ],
       [ 0.9026903 , -1.23462617, -0.70770647],
       [-1.69206178, -0.40836727, -0.40020384],
       [ 0.2576872 ,  1.5168972 , -0.76329561]])

In [4]:
# TESTING: PCA

skl_pca = PCA(n_components=2)
skl_pca.fit(raw_data)

my_pca = PrincipalComponentAnalysis(n_components=2)
my_pca.fit(raw_data)

display(
    skl_pca.transform(raw_data),
    my_pca.transform(raw_data),
)


array([[-0.38131303,  0.38839413],
       [-0.61587453,  3.37299852],
       [-4.33201691, -1.93358895],
       [ 0.47085344, -0.62204378],
       [ 4.85835104, -1.20575992]])

array([[-0.38131303,  0.38839413],
       [-0.61587453,  3.37299852],
       [-4.33201691, -1.93358895],
       [ 0.47085344, -0.62204378],
       [ 4.85835104, -1.20575992]])